In [51]:
# Installing packages in the notebook for conda if needed
#%pip install mne
#%pip install statsmodels
#%pip install scikit-learn
#%pip install researchpy
%pip install bioinfokit
# 

Note: you may need to restart the kernel to use updated packages.


In [52]:
# importing modules
import os
import pip
import numpy as np
import pandas as pd
import mne
from scipy import stats as st
import statistics as stats
from itertools import combinations
import researchpy as rp
from statsmodels.stats.multitest import multipletests as multi
from bioinfokit.analys import stat


In [53]:
epochs = mne.read_epochs('./data/Epochs/thatcher_epochs.fif')

Reading /Users/elisius/Library/CloudStorage/OneDrive-AarhusUniversitet/4. Semester/EXAM_Neuro/EEG_Thatcher_Effect/data/Epochs/thatcher_epochs.fif ...
    Found the data of interest:
        t =    -100.00 ...     696.00 ms
        0 CTF compensation matrices available


/var/folders/kf/wwp_17qs7v980_90pbjz9b240000gn/T/ipykernel_66823/2480100391.py:1: RuntimeWarning: This filename (./data/Epochs/thatcher_epochs.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs = mne.read_epochs('./data/Epochs/thatcher_epochs.fif')


Not setting metadata
358 matching events found
No baseline correction applied
0 projection items activated


In [54]:
# NOT evoked variabled 
all_changed = epochs['Changed']
all_unchanged = epochs['Unchanged']

# Not evoked Interactions
changed_0 = epochs['Changed/0']
changed_90 = epochs['Changed/90']
changed_180 = epochs['Changed/180']

unchanged_0 = epochs['Unchanged/0']
unchanged_90 = epochs['Unchanged/90']
unchanged_180 = epochs['Unchanged/180']

## familiar and unfamiliar
familiar = epochs['Familliar']
unfamiliar = epochs['Unfamilliar']

# Rotation Epochs:
# Not evoked Interactions
epoch_0 = epochs['0']
epoch_90 = epochs['90']
epoch_180 = epochs['180']

changed_0_familiar = epochs['Changed/0/Familliar']
changed_90_familiar = epochs['Changed/90/Familliar']
changed_180_familiar = epochs['Changed/180/Familliar']

unchanged_0_familiar = epochs['Unchanged/0/Familliar']
unchanged_90_familiar = epochs['Unchanged/90/Familliar']
unchanged_180_familiar = epochs['Unchanged/180/Familliar']

# ## Unfamiliar
changed_0_unfamiliar = epochs['Changed/0/Unfamilliar']
changed_90_unfamiliar = epochs['Changed/90/Unfamilliar']
changed_180_unfamiliar = epochs['Changed/180/Unfamilliar']

unchanged_0_unfamiliar = epochs['Unchanged/0/Unfamilliar']
unchanged_90_unfamiliar = epochs['Unchanged/90/Unfamilliar']
unchanged_180_unfamiliar = epochs['Unchanged/180/Unfamilliar']




# Evoked variables
## Changed vs unchaged
all_changed_evoked = epochs['Changed'].average()
all_unchanged_evoked = epochs['Unchanged'].average()

# INTERACTIONS
# Changed vs Unchanged X Inversion effect
## creating for changed vs unchanged for each rotation
# changed_0 = epochs['Changed/0'].average()
# changed_90 = epochs['Changed/90'].average()
# changed_180 = epochs['Changed/180'].average()

# unchanged_0 = epochs['Unchanged/0'].average()
# unchanged_90 = epochs['Unchanged/90'].average()
# unchanged_180 = epochs['Unchanged/180'].average()

# Familiar vs Unfamiliar X Changed vs Unchanged X Inversion effect
## creating for famiar vs unfamiliar interacting with inverse effect

# ## Familiar
# changed_0_familiar = epochs['Changed/0/Familliar'].average()
# changed_90_familiar = epochs['Changed/90/Familliar'].average()
# changed_180_familiar = epochs['Changed/180/Familliar'].average()

# unchanged_0_familiar = epochs['Unchanged/0/Familliar'].average()
# unchanged_90_familiar = epochs['Unchanged/90/Familliar'].average()
# unchanged_180_familiar = epochs['Unchanged/180/Familliar'].average()

# ## Unfamiliar
# changed_0_unfamiliar = epochs['Changed/0/Unfamilliar'].average()
# changed_90_unfamiliar = epochs['Changed/90/Unfamilliar'].average()
# changed_180_unfamiliar = epochs['Changed/180/Unfamilliar'].average()

# unchanged_0_unfamiliar = epochs['Unchanged/0/Unfamilliar'].average()
# unchanged_90_unfamiliar = epochs['Unchanged/90/Unfamilliar'].average()
# unchanged_180_unfamiliar = epochs['Unchanged/180/Unfamilliar'].average()

# Windowed mean
One of the simplest way in which we can determine whether the signal in our two conditions are statistically significant is by:

Segmenting our data using only certain channels in a specific time window. Keep in mind that which time window and channels should be established a priori, for instance according to the literature.
Taking the mean of that window across channels and and samples.
Running statistical tests on the windowed means from the two conditions.
In an experiment with multiple participants we would also average over trials from individual participants, in order to only have one data point per participant (and thereby avoid multiple comparisons). However, since we have one participant, we can keep one dimension of the individual data, i.e. the trials.

## T-test
We can now do a t-test on the trials from the two conditions, to establish whether the means of the two groups are statistically significant.

We can use the get_data() function to get the numerical values of the signal (in microvolts) for the t-test. tmin and tmax are used to define the size of the window, and the picks are the channels that we expect to see an effect in.


## Create ANOVA Test Function

In [55]:

def Anova_Epochs(Epoch_Condition, tmin, tmax, channels):
    n_conditions = range(len(Epoch_Condition))
    data = []
    for i in n_conditions:
        data.append(Epoch_Condition[i].get_data(picks=channels, tmin=tmin, tmax=tmax))
    
    means = []
    for i in n_conditions:
        mean_temp = np.mean(data[i], axis=2)
        means.append(np.mean(mean_temp, axis=1))
    
    res = stat()
    return st.f_oneway(*means)

# N170
## 'P7', 'P8', "TP9", "TP10", "O1", "O2"

In [56]:
print(Anova_Epochs(Epoch_Condition = [changed_180, unchanged_180], 
             tmin = .160, 
             tmax = .180,
             channels = ['P7', 'P8', "TP9", "TP10", "O1", "O2"]))### Default Threshold 

F_onewayResult(statistic=1.746618589914412, pvalue=0.19149185167655294)


In [7]:
print(Anova_Epochs(Epoch_Condition = [changed_0, unchanged_0], 
             tmin = .184, 
             tmax = .200,
             channels = ['P7', 'P8', "TP9", "TP10", "O1", "O2"]))### Default Threshold 

F_onewayResult(statistic=1.0793747198866743, pvalue=0.3031493051450015)


C:\Users\lauri\AppData\Local\Temp\ipykernel_20676\3370433452.py:6: RuntimeWarning: Channel(s) ['P9', 'P10'] could not be picked, because they are not present in the info instance.
  data.append(Epoch_Condition[i].get_data(picks=channels, tmin=tmin, tmax=tmax))
C:\Users\lauri\AppData\Local\Temp\ipykernel_20676\3370433452.py:6: RuntimeWarning: Channel(s) ['P9', 'P10'] could not be picked, because they are not present in the info instance.
  data.append(Epoch_Condition[i].get_data(picks=channels, tmin=tmin, tmax=tmax))


In [66]:
print(Anova_Epochs(Epoch_Condition = [familiar, unfamiliar], 
             tmin = .184, 
             tmax = .200,
             channels = ['TP9', 'TP10']))### Default Threshold 

F_onewayResult(statistic=1.4704976092717361, pvalue=0.22687415821269516)


## N170
"P7", "P8", "TP9", "TP10", "P9", "P10"

### Create T-Test Function

In [18]:
def Ttest_Epochs(Epoch_Condition, tmin, tmax, channels):
    data = []
    for i in range(2):
        data.append(Epoch_Condition[i].get_data(picks=channels, tmin=tmin, tmax=tmax))
    
    means = []
    for i in range(2):
        mean_temp = np.mean(data[i], axis=2)
        means.append(np.mean(mean_temp, axis=1))
    
    return st.ttest_ind(a=means[0], b=means[1])  

In [61]:
print(Ttest_Epochs(Epoch_Condition = [unchanged_0, changed_0], 
             tmin = .184, 
             tmax = .200,
             channels = ['P7', 'P8', "TP9", "TP10"]))

Ttest_indResult(statistic=-1.135051694926796, pvalue=0.2610226608748733)


In [36]:
print(Ttest_Epochs(Epoch_Condition = [unchanged_90, changed_90], 
             tmin = .184, 
             tmax = .200,
             channels = ['P7', 'P8', "TP9", "TP10"]))

Ttest_indResult(statistic=-0.9531761985875112, pvalue=0.34445646967791077)


In [37]:
print(Ttest_Epochs(Epoch_Condition = [unchanged_180, changed_180], 
             tmin = .184, 
             tmax = .200,
             channels = ['P7', 'P8', "TP9", "TP10"]))

Ttest_indResult(statistic=1.2375928034154984, pvalue=0.22085389905812924)


In [65]:
print(Ttest_Epochs(Epoch_Condition = [unchanged_0, unchanged_90, unchanged_180, changed_0, changed_90, changed_180], 
             tmin = .184, 
             tmax = .200,
             channels = ["TP9", "TP10"]))

Ttest_indResult(statistic=-0.8989995989803181, pvalue=0.3723700909867389)


### Pairwise T-tests in The Significant Window

In [22]:
def pairwise_Ttest(Epoch_Condition, tmin, tmax, channels, Epoch_Names):
    n_conditions = range(len(Epoch_Condition))
    data = []
    for i in n_conditions:
        data.append(Epoch_Condition[i].get_data(picks=channels, tmin=tmin, tmax=tmax))
    
    means = []
    for i in n_conditions:
        mean_temp = np.mean(data[i], axis=2)
        means.append(np.mean(mean_temp, axis=1))
    
    ## create empty lists
    pairwise_names = []
    pairwise_pval = []
    pairwise_tstat = []
    
    ## make all the combinations
    for a, b in combinations(Epoch_Names, 2): 
        y = f"{a} vs. {b}"
        pairwise_names.append(y)
        
    for a, b in combinations(means, 2): # function which make all combinations of 2 means possible
        y = st.ttest_ind(a, b)
        pairwise_pval.append(y.pvalue)
        pairwise_tstat.append(y.statistic)
    
    
    bonferroni = multi(pairwise_pval, alpha=0.05, method='bonferroni', is_sorted=False, returnsorted=False)
    
    ## create dataframe
    data = pd.DataFrame({"Comparison":pairwise_names, "P-Value":pairwise_pval, "t-statistic":pairwise_tstat, "Bonferroni Adj.":bonferroni[1]})
    
    return data
    

## Plotting different rotation for normal vs thatcher

In [59]:
## N170
pairwise = pairwise_Ttest(Epoch_Condition = [unchanged_0, unchanged_90, unchanged_180, changed_0, changed_90, changed_180], 
             tmin = .184, 
             tmax = .200,
             channels = ["TP9", "TP10"],
             Epoch_Names = ["Normal Upright", "Normal Ninety", "Normal Inverted", "Thatcher Upright", "Thatcher Ninety", "Thatcher Inverted"]
              )

print("T test on different rotations for Normal vs thatcher images")
pairwise

T test on different rotations for Normal vs thatcher images


,Comparison,P-Value,t-statistic,Bonferroni Adj.
0,Normal Upright vs. Normal Ninety,0.372370,-0.899000,1.0
1,Normal Upright vs. Normal Inverted,0.269234,-1.115508,1.0
2,Normal Upright vs. Thatcher Upright,0.514689,-0.655576,1.0
3,Normal Upright vs. Thatcher Ninety,0.287442,-1.073615,1.0
4,Normal Upright vs. Thatcher Inverted,0.893952,0.133892,1.0
5,Normal Ninety vs. Normal Inverted,0.931723,0.086050,1.0
6,Normal Ninety vs. Thatcher Upright,0.661535,0.440052,1.0
7,Normal Ninety vs. Thatcher Ninety,0.973968,-0.032773,1.0
8,Normal Ninety vs. Thatcher Inverted,0.332200,0.977869,1.0
9,Normal Inverted vs. Thatcher Upright,0.624790,0.491696,1.0


## Plotting different familiarities at different rotations for normal vs thatcher

In [62]:
## N170
pairwise = pairwise_Ttest(Epoch_Condition = [unchanged_0_familiar, unchanged_180_familiar, changed_0_familiar, changed_180_familiar, unchanged_0_unfamiliar, unchanged_180_unfamiliar, changed_0_unfamiliar, changed_180_unfamiliar], 
             tmin = .184, 
             tmax = .200,
             channels = ["TP9", "TP10"],
             Epoch_Names = ["Familiar Normal Upright", "Familiar Normal Inverted", "Familiar Thatcher Upright", "Familiar Thatcher Inverted", "Unfamiliar Normal Upright", "Unfamiliar Normal Inverted", "Unfamiliar Thatcher Upright", "Unfamiliar Thatcher Inverted"]
              )

print("T test on different rotations for Normal vs thatcher images")
pairwise

T test on different rotations for Normal vs thatcher images


,Comparison,P-Value,t-statistic,Bonferroni Adj.
0,Familiar Normal Upright vs. Familiar Normal In...,0.280811,-1.099741,1.0
1,Familiar Normal Upright vs. Familiar Thatcher ...,0.401794,-0.851356,1.0
2,Familiar Normal Upright vs. Familiar Thatcher ...,0.824647,0.223658,1.0
3,Familiar Normal Upright vs. Unfamiliar Normal ...,0.565792,-0.581144,1.0
4,Familiar Normal Upright vs. Unfamiliar Normal ...,0.246600,-1.183403,1.0
5,Familiar Normal Upright vs. Unfamiliar Thatche...,0.448654,-0.768448,1.0
6,Familiar Normal Upright vs. Unfamiliar Thatche...,0.510832,-0.666046,1.0
7,Familiar Normal Inverted vs. Familiar Thatcher...,0.841290,0.202113,1.0
8,Familiar Normal Inverted vs. Familiar Thatcher...,0.260706,1.147941,1.0
9,Familiar Normal Inverted vs. Unfamiliar Normal...,0.713132,0.371405,1.0
